## Flask

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from flask import Flask, request, jsonify
import tensorflow as tf
import time
import json
import gc
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from threading import Thread
import logging

app = Flask(__name__)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Variables globales pour le modèle et un indicateur de disponibilité
model = None
model_ready = False

# Fonction pour charger le modèle
def load_model():
    global model, model_ready
    try:
        logger.info("Starting model load in a separate thread...")
        start_time = time.time()
        model = tf.keras.models.load_model('sentiment_lstm_model.keras')
        model_load_time = time.time() - start_time
        logger.info(f"Model loaded successfully in {model_load_time:.2f} seconds.")
        model_ready = True
    except Exception as e:
        logger.error(f"Error loading model: {e}")
        model_ready = False

# Démarrer le chargement du modèle dans un thread
load_thread = Thread(target=load_model)
load_thread.start()

# Attendre que le modèle soit chargé
load_thread.join()

# Chargement du tokenizer
try:
    with open('tokenizer.json') as f:
        data = json.load(f)
        tokenizer = tokenizer_from_json(data)
        logger.info("Tokenizer loaded successfully.")
except Exception as e:
    logger.error(f"Error loading tokenizer: {e}")

# Prétraitement du texte
def preprocess_text(text, max_len=60):
    logger.info(f"Original text: {text}")
    seq = tokenizer.texts_to_sequences([text])
    logger.info(f"Tokenized sequence: {seq}")
    padded = pad_sequences(seq, maxlen=max_len)
    return padded

@app.route('/predict', methods=['POST'])
def predict():
    if not model_ready:
        return jsonify({'error': 'Model is not loaded yet. Please try again later.'}), 503

    try:
        data = request.get_json(force=True)
        if 'text' not in data:
            return jsonify({'error': 'Missing "text" field in request data'}), 400

        logger.info(f"Received data: {data}")

        preprocessed_text = preprocess_text(data['text'])
        logger.info(f"Preprocessed text: {preprocessed_text}")

        prediction_start_time = time.time()
        prediction = model.predict(preprocessed_text)
        prediction_time = time.time() - prediction_start_time
        logger.info(f"Prediction time: {prediction_time:.2f} seconds")
        logger.info(f"Prediction: {prediction}")

        sentiment = "Positif" if prediction[0][0] > 0.5 else "Négatif"
        gc.collect()

        return jsonify({
            "score": float(prediction[0][0]),
            "sentiment": sentiment
        })
    except Exception as e:
        logger.error(f"Error in predict function: {e}")
        return jsonify({'error': str(e)}), 400

if __name__ == '__main__':
    try:
        app.run(debug=True)
    except SystemExit:
        pass


INFO:__main__:Starting model load in a separate thread...
INFO:__main__:Model loaded successfully in 0.44 seconds.
INFO:__main__:Tokenizer loaded successfully.


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


INFO:werkzeug: * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
INFO:werkzeug: * Restarting with stat


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Définir le nombre maximum de mots dans le tokenizer
max_words = 15000
max_len = 60

# Charger le modèle pré-entraîné
model = tf.keras.models.load_model('sentiment_lstm_model.keras')

# Charger et configurer le tokenizer avec le même vocabulaire que celui utilisé lors de l'entraînement
tokenizer = Tokenizer(num_words=max_words)
# Supposez que tokenizer soit déjà ajusté sur le texte d'entraînement

def preprocess_text(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post', truncating='post')
    return padded_sequence

test_tweets = [
    "I can't believe how terrible this service is. Absolutely awful!",
    "Worst experience ever. Totally disappointing and frustrating.",
    "Everything about this product is just so bad. Waste of money.",
    "I'm really not happy with how things turned out. Expected better.",
    "Not impressed with the quality at all. Quite disappointing.",
    "This is not what I ordered. Very misleading.",
    "I was hoping for more, but it's just okay. Not quite what I wanted.",
    "It's alright, but it could be a lot better. Needs improvement.",
    "Service was slow and unresponsive, but the product was decent.",
    "It’s fine, nothing too special but not too bad either.",
    "I had an average experience, nothing to write home about.",
    "It’s okay, does the job but doesn’t exceed expectations.",
    "Pretty good overall, just a few minor issues here and there.",
    "Decent product for the price, happy with the purchase.",
    "Met most of my expectations, would recommend with some reservations.",
    "Really satisfied with this. It’s just what I needed.",
    "Great product, would definitely buy again. Very happy.",
    "Good service and quality, met my expectations well.",
    "Absolutely love this! Exceeded all my expectations.",
    "Best purchase I've made in a while. Highly recommend!",
    "Fantastic experience from start to finish. Couldn't be happier!"
]

for tweet in test_tweets:
    preprocessed_text = preprocess_text(tweet)
    prediction = model.predict(preprocessed_text)
    sentiment = "Positif" if prediction[0][0] > 0.5 else "Négatif"
    print(f"Tweet: {tweet}")
    print(f"Score: {prediction[0][0]:.4f}, Sentiment: {sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 513ms/step
Tweet: I can't believe how terrible this service is. Absolutely awful!
Score: 0.2567, Sentiment: Négatif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Tweet: Worst experience ever. Totally disappointing and frustrating.
Score: 0.2567, Sentiment: Négatif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Tweet: Everything about this product is just so bad. Waste of money.
Score: 0.2567, Sentiment: Négatif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Tweet: I'm really not happy with how things turned out. Expected better.
Score: 0.2567, Sentiment: Négatif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Tweet: Not impressed with the quality at all. Quite disappointing.
Score: 0.2567, Sentiment: Négatif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Tweet: This is not what I ordered. Very misleading.
Score: 0.2567, Sentiment: Négatif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Tweet: I was hoping for more, but it's just okay. Not quite what I wanted.
Score: 0.2567, Sentiment: Négatif
1/1 ━━━━━━━━━━━━━━━

In [3]:
import requests

url = 'https://projet7oc.azurewebsites.net/predict'

# Inclure à la fois 'text' et 'true_label' dans les données de la requête
data = {
    'text': 'Here is a sample tweet text to analyze.',
    'true_label': '1'  # ou 'positive' ou 'negative' selon ce que le serveur attend
}

response = requests.post(url, json=data)
print(response.json())


{'score': 0.2705337703227997, 'sentiment': 'Négatif'}
